In [16]:
import polars as pl
import numpy as np
import pandas as pd
import os

# df_metrics = df_metrics.group_by("dataset").agg(
#     [
#         (pl.col(col).mean().round(3).cast(pl.Utf8) + " ± " + 
#          pl.col(col).std().round(3).cast(pl.Utf8)).alias(col)
#         for col in df_metrics.columns if col != "dataset"
#     ]
# )

def calculate_metrics_from_confusion_matrix(matrix_path):
    """
    Calculates ACC, Sn, Sp, F1, and MCC from a binary confusion matrix CSV.
    """
    df = pd.read_csv(matrix_path, index_col=0)

    if len(df.columns) < 4:
        # Extract values (assuming order like your example)
        TN = df.loc["negative", "negative"]
        FP = df.loc["negative", "positive"]
        FN = df.loc["positive", "negative"]
        TP = df.loc["positive", "positive"]

        # Metrics
        ACC = (TP + TN) / (TP + TN + FP + FN)
        Sn = TP / (TP + FN) if (TP + FN) > 0 else 0  # Sensitivity (Recall)
        Sp = TN / (TN + FP) if (TN + FP) > 0 else 0  # Specificity
        F1 = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0
        MCC = ((TP * TN) - (FP * FN)) / np.sqrt(
            (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)
        ) if all(x > 0 for x in [(TP + FP), (TP + FN), (TN + FP), (TN + FN)]) else 0

        return {
            "ACC_test": ACC,
            "Sn_test": Sn,
            "Sp_test": Sp,
            "F1_test": F1,
            "MCC_test": MCC,
        }
    else:
        return {
            "ACC_test": None,
            "Sn_test": None,
            "Sp_test": None,
            "F1_test": None,
            "MCC_test": None,
        }

# ---------- Main pipeline ----------
pl.Config(tbl_rows=50)

full_datasets_path = "App/datasets"
datasets_list = [
    os.path.join(full_datasets_path, item)
    for item in os.listdir(full_datasets_path)
    if os.path.isdir(os.path.join(full_datasets_path, item))
]

df_metrics = pd.DataFrame()

for dataset_path in datasets_list:
    experiments_folder = os.path.join(dataset_path, "runs")
    if os.path.exists(experiments_folder):
        runs_folders = [
            os.path.join(experiments_folder, run_folder)
            for run_folder in os.listdir(experiments_folder)
        ]

        for run_folder in runs_folders:
            # --- Load training metrics ---
            train_path = os.path.join(run_folder, "training_kfold(10)_metrics.csv")
            if not os.path.exists(train_path):
                continue

            df_metrics_run = pd.read_csv(train_path)
            df_metrics_run["dataset"] = dataset_path.split("/")[-1]

            # --- Load test confusion matrix & compute metrics ---
            test_confusion_path = os.path.join(run_folder, "test_confusion_matrix.csv")
            test_other_path = os.path.join(run_folder, "metrics_other.csv")
            if os.path.exists(test_other_path):
                test_metrics = calculate_metrics_from_confusion_matrix(test_confusion_path)
                for k, v in test_metrics.items():
                    df_metrics_run[k] = v

                df_metrics_run["AUC_test"] = pl.read_csv(test_other_path).filter(pl.col("Metric") == "AUC")["Value"].item()
            else:
                # If missing, fill with NaN
                for k in ["ACC_test", "Sn_test", "Sp_test", "F1_test", "MCC_test"]:
                    df_metrics_run[k] = np.nan

            df_metrics = pd.concat([df_metrics, df_metrics_run], ignore_index=True)

# ---------- Sorting and Polars conversion ----------
metric_columns = [
    column for column in df_metrics.columns.drop(["dataset"]).tolist() if "std" not in column
]
df_metrics = pl.from_pandas(df_metrics[["dataset"] + metric_columns]).sort(by=["dataset"])

# Optional sorting by dataset number if datasets are named like "dataset1", "dataset2", etc.
df_sorted = (
    df_metrics.with_columns(
        pl.col("dataset").str.extract(r"dataset(\d+)").cast(pl.Int64).alias("dataset_num")
    )
    .sort("dataset_num")
    .drop("dataset_num")
)

df_sorted.select(["dataset", "Sn", "Sp", "ACC", "MCC", "AUC", "Sn_test", "Sp_test", "ACC_test", "MCC_test", "AUC_test"]).write_csv("runs.csv")

In [ ]:
df_papers = pl.DataFrame({"dataset": ["dataset1_zhang_protein"], 
             "ACC": [0.871], 
             "MCC": [None], 
             "F1": [None], 
             "balanced_ACC": [None], 
             "kappa": [None], 
             "gmean": [None], 
             "F1_micro": [None], 
             "F1_macro": [None], 
             "F1_w": [None]})
df_papers

dataset,ACC,MCC,F1,balanced_ACC,kappa,gmean,F1_micro,F1_macro,F1_w
str,null,null,null,null,null,null,null,null,null
"""dataset1_zhang_protein""",null,null,null,null,null,null,null,null,null
